Boston Crime Zone Heatmap made with Python and Folium
==========================================

In [ ]:
import folium
boston_lat_lon = [ 42.302, -71.1500 ]
m = folium.Map(
    location=boston_lat_lon,
    zoom_start=11,
    tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
    attr='Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; <a href="https://opentopomap.org">OpenTopoMap</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'
)
m

Import data from file

In [ ]:
import pandas as pd
df = pd.read_csv("boston_crime.csv")


Access dataframe values

In [ ]:
df.head()

Rename columns.

In [ ]:
columns = {
    'OCCURRED_ON_DATE': 'date',
    'OFFENSE_CODE_GROUP': 'offense',
    'SHOOTING': 'shooting',
    'Lat': 'lat',
    'Long': 'lon',
}
df = df.rename(columns=columns)
df = df[ list(columns.values()) ]


Deal with data types

In [ ]:
type(df.date[0])
df.date = pd.to_datetime(df.date)
df = df.sort_values(by='date')
print(df.date[0:10])

Deal with null values.

In [ ]:
df.shooting = (df.shooting == "Y")
df = df.dropna()
df.head()

Group data by month

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta

months = []
start = datetime.datetime(2015, 6, 1)
while start < df.date.max():
    end = start + relativedelta(months=+1)
    mask = (start <= df.date) & (df.date < end)
    df_month = df[mask]
    df_month = df_month[ ['lat', 'lon']]
    months.append(df_month)
    start = end
print(months[0])

Create heatmap

In [ ]:
from folium.plugins import HeatMapWithTime

m = folium.Map(boston_lat_lon, zoom_start=11)
hm = HeatMapWithTime(
    data=[ m.values.tolist() for m in months ],
    radius=5,
    max_opacity=0.5,
    auto_play=False,
)
hm.add_to(m)
m

Spatially clustered data

In [ ]:
LAT_LON_GRID = 0.005

def custom_round(val, resolution):
    return round(val / resolution) * resolution

def cluster(df_interval):
    data = df_interval.copy()
    data = custom_round(data, LAT_LON_GRID)
    data = data.groupby(["lat", "lon"]).size().reset_index(name="weight")
    data.weight = data.weight / data.weight.max()
    return data

start = datetime.datetime(2015, 6, 1)

end = start + relativedelta(months=+1)
mask = (start <= df.date) & (df.date < end)
df_month = df[mask]
df_month = df_month[ ['lat', 'lon']]

print(cluster(df_month))

Redraw heat map

In [ ]:
from folium.plugins import HeatMapWithTime

m = folium.Map(boston_lat_lon, zoom_start=11)
hm = HeatMapWithTime(
    data=[ cluster(m).values.tolist() for m in months ],
    radius=15,
    max_opacity=0.5,
    auto_play=False,
)
hm.add_to(m)
m